In [6]:
import pystan


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_fbfb2c0a1a7cb8af07b24a70f161a334 NOW.


Inference for Stan model: anon_model_fbfb2c0a1a7cb8af07b24a70f161a334.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu          1.4    0.04   0.58   0.39   0.96   1.37   1.81   2.55  231.0   1.01
phi[0]    -0.44    0.03   0.41  -0.99  -0.76  -0.51  -0.21   0.56  175.0   1.01
phi[1]    -0.27    0.02    0.3  -0.84  -0.47  -0.26  -0.07    0.3  172.0   1.01
theta[0]    0.3    0.03   0.41  -0.69   0.04   0.35   0.61   0.97  156.0   1.01
theta[1]   0.06    0.03   0.34  -0.53  -0.18   0.03   0.29   0.77  168.0   1.01
sigma       2.0  9.9e-3    0.2   1.65   1.85   1.98   2.13   2.42  409.0   1.01
lp__     -72.33     0.1   1.95 -76.98 -73.47 -71.97 -70.89 -69.57  365.0   1.01

Samples were drawn using NUTS at Thu Aug 10 21:00:30 2017.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction 

In [ ]:

code = '''
data {
    int<lower=1> T;
    real y[T];
}
parameters {
    real mu;
    vector<lower = -1, upper = 1>[2] phi;
    vector<lower = -1, upper = 1>[2] theta;
    real<lower=0> sigma;
} 
model {
    vector[T] nu;
    vector[T] err;
    nu[1] = mu + phi[1] * mu;
    err[1] = y[1] - nu[1];
    nu[2] = mu + phi[1]*y[1] + theta[1]*err[1];
    err[2] = y[2] - nu[2];
    for (t in 3:T) {
        nu[t] = mu + phi[1] * y[t-1] + phi[2]*y[t-2] 
              + theta[1] * err[t-1] + theta[2] * err[t-2];
        err[t] = y[t] - nu[t];
    }
    mu ~ normal(0, 10);
    phi ~ normal(0, 2);
    theta ~ normal(0, 2);
    sigma ~ cauchy(0, 5);
    err ~ normal(0, sigma);
}
'''
y =  [ 0.596,  2.127,  3.093,  0.423, -1.508,  3.28 , -5.571,  2.623,
        1.335, -1.234, -0.364,  0.081, -1.076, -0.791,  1.715,  1.266,
        0.566, -1.333,  1.871,  2.429, -2.064, -0.336,  1.407,  2.787,
        2.743, -3.073,  1.335,  1.163,  2.982,  4.317,  2.913,  0.432,
       -3.238,  1.785,  2.766,  2.252,  1.024,  2.167, -0.334, -0.106,
        1.959, -0.784,  0.387,  2.816, -0.762, -1.897,  1.99 , -0.965,
        2.714,  1.237,  0.701,  3.814, -1.459,  1.021,  4.114,  0.038,
       -1.391,  2.91 ]
data = {
    'T': len (y),
    'y': y
}

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=4)
# params = fs = fit.summary()['summary']
print fit

In [10]:

code = '''
data {
    int<lower=1> T;
    real y[T];
}
parameters {
    real mu;
    vector<lower = -1, upper = 1>[2] phi;
    vector<lower = -1, upper = 1>[2] theta;
    real<lower=0> sigma;
    real y_tilda;
} 
model {
    vector[T] nu;
    vector[T] err;
    nu[1] = mu + phi[1] * mu;
    err[1] = y[1] - nu[1];
    nu[2] = mu + phi[1]*y[1] + theta[1]*err[1];
    err[2] = y[2] - nu[2];
    for (t in 3:T) {
        nu[t] = mu + phi[1] * y[t-1] + phi[2]*y[t-2] 
              + theta[1] * err[t-1] + theta[2] * err[t-2];
        err[t] = y[t] - nu[t];
    }
    mu ~ normal(0, 10);
    phi ~ normal(0, 2);
    theta ~ normal(0, 2);
    sigma ~ cauchy(0, 5);
    err ~ normal(0, sigma);
    y_tilda ~ normal(mu + y[T]*phi[1]+y[T-1]*phi[2] + theta[1]*err[T] + theta[2]*err[T-1], sigma);
}

'''
y =  [ 0.596,  2.127,  3.093,  0.423, -1.508,  3.28 , -5.571,  2.623,
        1.335, -1.234, -0.364,  0.081, -1.076, -0.791,  1.715,  1.266,
        0.566, -1.333,  1.871,  2.429, -2.064, -0.336,  1.407,  2.787,
        2.743, -3.073,  1.335,  1.163,  2.982,  4.317,  2.913,  0.432,
       -3.238,  1.785,  2.766,  2.252,  1.024,  2.167, -0.334, -0.106,
        1.959, -0.784,  0.387,  2.816, -0.762, -1.897,  1.99 , -0.965,
        2.714,  1.237,  0.701,  3.814, -1.459,  1.021,  4.114,  0.038,
       -1.391,  2.91 ]
data = {
    'T': len (y),
    'y': y
}

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=4)
# params = fs = fit.summary()['summary']
print fit

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_6abda944bb0b6acb21d3626f5be8dfef NOW.


Inference for Stan model: anon_model_6abda944bb0b6acb21d3626f5be8dfef.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu         1.37    0.04   0.57   0.44   0.95   1.31   1.74   2.67  239.0   1.01
phi[0]    -0.44    0.03   0.43  -0.99   -0.8  -0.52  -0.16   0.59  187.0   1.01
phi[1]    -0.25    0.02   0.28  -0.82  -0.43  -0.24  -0.06   0.25  270.0    1.0
theta[0]   0.28    0.03   0.44  -0.76 2.8e-3   0.36   0.63   0.91  191.0   1.01
theta[1]   0.04    0.02   0.32  -0.48  -0.19   0.01   0.24   0.73  216.0   1.01
sigma      1.98  8.5e-3   0.19   1.61   1.86   1.97    2.1   2.39  515.0   1.01
y_tilda    0.78    0.08   2.03  -3.09  -0.68   0.78   2.25   4.64  662.0    1.0
lp__     -73.37    0.13   2.19 -78.61 -74.52 -73.07 -71.74 -70.37  305.0   1.01

Samples were drawn using NUTS at Thu Aug 10 22:32:39 2017.
For each parameter, n_eff is a cr